In [ ]:
import pickle
from GP_regression import * 
import scipy.stats as st
from scipy.optimize import minimize

In [ ]:
GP_titles = ['GP_calib_best/log_L_P1_GP.pickle',\
             'GP_calib_best/log_L_P2_GP.pickle',\
             'GP_calib_best/log_L_P3_GP.pickle'] 
profiles = [1, 2, 3]

GPs = {}

for profile, title in zip(profiles, GP_titles):
    
    GPs[profile] = GPy.load(title)

In [ ]:
opt_val = {}
vals_res = {}
opt_res = {}

## nominal values fig 3a

opt_val[1] = np.array([[0.2715, 0.6257, 0.3933, 0.6737, 0.3185]])
vals_res[1] = -187.72038344
opt_res[1] = -107.05880716



## nominal values fig 3b

opt_val[2] = np.array([[0.0529825, 0.457069 , 0.306745 , 0.757327 , 0.1542485]])
vals_res[2] = -219.01617537
opt_res[2] = -110.03513997


## nominal values fig 4

opt_val[3] = np.array([[0.0315321, 0.3624473, 0.1819143, 0.9467015, 0.0571799]])
vals_res[3] = -397.16571399
opt_res[3] = -54.5520279

In [ ]:
X_LH = np.load('GP_calib_best/LH_freyja_samples.npy')

results = {}
    
results[1] = np.load('GP_calib_best/Log_L_samples_P1.npy')
results[2] = np.load('GP_calib_best/Log_L_samples_P2.npy')
results[3] = np.load('GP_calib_best/Log_L_samples_P3.npy')




In [ ]:
X = {}
Z = {}
var = {}

for profile in profiles:
    
    X[profile] = np.r_[opt_val[profile], X_LH]
    
    res = np.r_[opt_res[profile], results[profile]]
    
    Z[profile] = res[:, None]
    
    var[profile] = np.r_[np.abs(0.05*opt_res[profile]), np.zeros(X_LH.shape[0])]
    
    

In [ ]:
X_s = {}

for profile in profiles:
    
    x = X[profile]
    
    min_vals = 0.5 * opt_val[profile][0]; max_vals = 1.5 * opt_val[profile][0]
    scaled = np.zeros_like(x)
    

    for j in range(X_LH.shape[1]):
        
        scaled[:, j] = (x[:, j] - min_vals[j]) / (max_vals[j] - min_vals[j])
    
    X_s[profile] = scaled
    
# print(X_s)

In [ ]:
priors = {}

for profile in profiles:
    
    priors[profile] = [GPy.priors.Gaussian(0.5, 0.5) for x in X_s[profile]]
    

In [ ]:
def build_gp(x, profile):
    X_s[profile][0, :] = x[:N_param] # set calibration parameters
    kern = GPy.kern.Exponential(input_dim=X_s[profile].shape[1], ARD=True)
    m = GPy.models.GPHeteroscedasticRegression(X_s[profile], Z[profile], kern)
    m.likelihood.variance[:, 0] = var[profile] # fix variances at known values
    m.likelihood.fix()
    m.optimizer_array = m0.optimizer_array # fix hyperparameters at optimized values
    return m

def log_param_priors(x, profile):
    ps = [p.lnpdf(_x) for (_x, p) in zip(x, priors[profile])]
    return np.sum(ps)

def log_post(x, profile):
    m = build_gp(x, profile)
    return m.log_likelihood() + m.log_prior() + log_param_priors(x, profile)

In [ ]:
def grw_metropolis_mcmc(x0, log_h, N, sigma, profile):
    x0 = np.array(x0)
    assert x0.ndim == 1
    d = x0.shape[0]
    X = np.zeros((N+1, d))
    X[0, :] = x0
    accepted = 0
    log_hp = log_h(x0, profile) # previous value of log(h(x))
    
    for i in range(1, N+1):
        # generation
        Xn = X[i-1,:] + sigma*np.random.randn(d)
        
        # calculation 
        log_hn = log_h(Xn, profile)
        alpha = min(1, np.exp(log_hn - log_hp))
        
        # accept/reject
        if np.random.rand() <= alpha:
            X[i, :] = Xn        # accept
            log_hp = log_hn
            accepted += 1
        else:
            X[i, :] = X[i-1, :] # reject
            
    return X, accepted / N

In [ ]:
profile = 3

N_param = 5
N_hyper = 0 # len(m0.optimizer_array)
x0 = np.ones(N_param + N_hyper)
x0[:N_param] = 0.5 # start with the unmodified parameters

X_s[profile][0, :] = x0[:N_param] # set calibration parameters
kern = GPy.kern.Exponential(input_dim=X_s[profile].shape[1], ARD=True)
m0 = GPy.models.GPHeteroscedasticRegression(X_s[profile], Z[profile], kern)
m0.likelihood.variance[:, 0] = var[profile] # fix variances at known values
m0.likelihood.fix()
m0.optimize_restarts(5)


In [ ]:
# for sigma_p in [0.1, 0.2, 0.5]:
#     chain, accept = grw_metropolis_mcmc(x0, log_post, 1000, sigma_p, profile=1)
#     print(sigma_p, accept, x0)


In [ ]:
chain, accept = grw_metropolis_mcmc(x0, log_post, 10000, 0.5, profile = profile)
print('acceptance ratio:', accept)

In [ ]:
parameter_names = [r'$f_e$', r'$f_i$',r'$RA$', r'$C$', r'$LPI$']

plt.plot(chain[:]);
plt.legend(parameter_names);

In [ ]:
print(len(chain))

In [ ]:
burn = 3000
thin = 40
# log_L_calib = []
# for sample in chain[burn::thin]:
#     m = build_gp(sample, profile=1)
#     Z_p, V_p = m.predict(X_s[1],  Y_metadata={'output_index': np.arange(len(var[1]))[:,None]})
#     log_L_calib.append(Z_p[0, 0])
# log_L_calib = np.array(log_L_calib)

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(12,4))
for i, ax in enumerate(axs.flat):
    if i >= 5: continue
    ax.acorr(chain[burn::thin, i], detrend=plt.mlab.detrend_mean, maxlags=50, color='C0')
    ax.set_xlim(0, 50)
    ax.hlines([0.5], 0, 50, color='k', ls='dashed')
    ax.set_title(f'$x_{i+1}$ (label)')
fig.subplots_adjust(wspace=0.5)


In [ ]:
fig, axs = plt.subplots(2, 3)
x = np.linspace(0, 1)

min_vals = 0.5 * opt_val[profile][0]; max_vals = 1.5 * opt_val[profile][0]
for j in range(5):
    ax = axs.flat[j]
    x = np.linspace(0.99*min_vals[j], 1.01*max_vals[j], 500)
    sns.histplot(chain[burn::thin, j]*(max_vals[j]-min_vals[j]) + min_vals[j], 
                 stat='density', kde=True, color='C1', ax=ax)
    u = st.uniform(min_vals[j], max_vals[j]-min_vals[j])
    ax.plot(x, u.pdf(x), ls='-')    
    ax.set_title(parameter_names[j])
    ax.axvline(opt_val[profile][0][j], ls='--', lw=2)
    ax.set_xlabel(r'Value')
plt.subplots_adjust(wspace=0.45, hspace=0.3)
axs.flat[-1].set_visible(False);
plt.show()

In [ ]:
# import pandas as pd
# pg = sns.pairplot(pd.DataFrame(chain, columns=parameter_names), plot_kws={'s': 10})
# pg.map_lower(sns.kdeplot);

In [ ]:
fig = plt.figure()

colours = ['C0', 'C1', 'C2', 'C3', 'C4']

index = [0, 1, 2, 3, 4]

peaks = []

for j, C in zip(index, colours):
   
    ax = sns.kdeplot(chain[burn::thin, j]*(max_vals[j]-min_vals[j]) + min_vals[j], color=C, label = parameter_names[j], lw  = 3)        
    # plt.axvline(opt_val[profile][0][j], ls='--', lw=2, color = C, label = r'Optimal GP ' + parameter_names[j])
    plt.xlabel(r'Value', fontsize = 18)
    plt.ylabel(r'Density', fontsize = 18)
    plt.title(r'Profile = '+str(profile), fontsize = 20)
    plt.legend(fontsize = 18)
    plt.xticks(fontsize = 15)
    plt.yticks(fontsize = 15)
    x = ax.lines[j].get_xdata() # Get the x data of the distribution
    y = ax.lines[j].get_ydata() # Get the y data of the distribution
    maxid = np.argmax(y) # The id of the peak (maximum of y data)
    peaks.append(x[maxid])
plt.show()

In [ ]:
fig = plt.figure()

colours = ['C0', 'C1', 'C2', 'C3', 'C4']

index = [0, 1, 2, 3, 4]



for j, C in zip(index, colours):
   
    ax = sns.kdeplot(chain[burn::thin, j]*(max_vals[j]-min_vals[j]) + min_vals[j], color=C, label = parameter_names[j] + r' (' + str(np.round(peaks[j],3)) + r')', lw  = 3)        
    # plt.axvline(opt_val[profile][0][j], ls='--', lw=2, color = C, label = r'Optimal GP ' + parameter_names[j])
    plt.xlabel(r'Value', fontsize = 18)
    plt.ylabel(r'Density', fontsize = 18)
    plt.title(r'Profile = '+str(profile), fontsize = 20)
    plt.legend(fontsize = 18)
    plt.xticks(fontsize = 15)
    plt.yticks(fontsize = 15)
#     plt.axvline(x = peaks[j], color = C, lw =3, linestyle = '--')
#     plt.axvline(x =0.06, color = 'black')

plt.xlim(-0.1, 1.1)
plt.show()

In [ ]:
print(peaks)